# Experiments with Toy (synthetic) dataset

Generate data

$E \sim N(0, \sigma_e^2 I)$

for each $k$

$R_k \sim MN(0, \sigma_r^2 I, I)$

$X_k \sim N(ER_kE^\top, \sigma_x^2)$

and fit the models with this dataset.

In [1]:
import numpy as np
import logging
import itertools
from scipy.sparse import csr_matrix
import rescal
from brescal import BayesianRescal

%matplotlib inline  
logger = logging.getLogger()
logger.setLevel(logging.INFO)

n_entity = 10
n_dim = 5
n_relation = 20

var_e = 1.
var_r = 1.
var_x = 1.
e_mean = np.zeros(n_dim)
r_mean = np.zeros(n_dim**2)
E = np.random.multivariate_normal(e_mean, np.identity(n_dim) * var_e, size = n_entity)
R = np.zeros([n_relation, n_dim, n_dim])
for k in range(n_relation):
    R[k] = np.random.multivariate_normal(r_mean, np.identity(n_dim**2)*var_r).reshape(n_dim,n_dim)
X = np.zeros([n_relation, n_entity, n_entity])
for k in range(n_relation):
    ERET = np.dot(np.dot(E, R[k]), E.T)
    for i,j in itertools.product(range(n_entity), repeat=2):
        X[k,i,j] = np.random.normal(ERET[i,j], var_x)

In [2]:
import itertools
trainX = np.zeros_like(X)
p = 1
for k in range(n_relation):
    for i,j in itertools.product(range(n_entity),repeat=2):
        if X[k, i, j] and np.random.binomial(1, p):
            trainX[k, i, j] = X[k, i, j]

In [3]:
model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r)
model.fit(trainX, max_iter=10)

INFO:brescal:[  0] LL: -10096.246 | fit: 0.35320 |  sec: 0.020
INFO:brescal:[  1] LL: -1687.381 | fit: 0.91687 |  sec: 0.020
INFO:brescal:[  2] LL: -1406.120 | fit: 0.93255 |  sec: 0.020
INFO:brescal:[  3] LL: -1380.027 | fit: 0.93100 |  sec: 0.022
INFO:brescal:[  4] LL: -1363.825 | fit: 0.93096 |  sec: 0.021
INFO:brescal:[  5] LL: -1342.647 | fit: 0.93152 |  sec: 0.021
INFO:brescal:[  6] LL: -1336.328 | fit: 0.93045 |  sec: 0.023
INFO:brescal:[  7] LL: -1311.620 | fit: 0.93008 |  sec: 0.023
INFO:brescal:[  8] LL: -1248.554 | fit: 0.93404 |  sec: 0.023
INFO:brescal:[  9] LL: -1300.673 | fit: 0.93093 |  sec: 0.024


In [4]:
csr_X = list()
for k in range(n_relation):
    csr_X.append(csr_matrix(trainX[k]))
    
E, R, f, itr, exectimes = rescal.rescal_als(csr_X, n_dim)

INFO:RESCAL:[  0] fit: 0.94523 | delta: 9.5e-01 | secs: 0.00836
INFO:RESCAL:[  1] fit: 0.95052 | delta: 5.3e-03 | secs: 0.01008
INFO:RESCAL:[  2] fit: 0.95056 | delta: 3.7e-05 | secs: 0.00966


In [5]:
_X = model._reconstruct()

print("BayesRESCAL:Element-wise squared error: %.3f" %(np.sum((X-_X)**2)))

_X = np.zeros_like(X)
for k in range(n_relation):
    _X[k] = np.dot(np.dot(E, R[k]), E.T)
    
print("RESCAL:Element-wise squared error: %.3f" %(np.sum((X-_X)**2)))

BayesRESCAL:Element-wise squared error: 2039.106
RESCAL:Element-wise squared error: 1459.629


## Split data into training/test data

... but both models consider the input tensor as a full tensor.

In [6]:
import itertools
trainX = np.zeros_like(X)
p = 0.5
for k in range(n_relation):
    for i,j in itertools.product(range(n_entity),repeat=2):
        if X[k, i, j] and np.random.binomial(1, p):
            trainX[k, i, j] = X[k, i, j]
model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r)
model.fit(trainX, max_iter=10)            

INFO:brescal:[  0] LL: -5724.418 | fit: 0.15797 |  sec: 0.030
INFO:brescal:[  1] LL: -3229.894 | fit: 0.53372 |  sec: 0.025
INFO:brescal:[  2] LL: -2748.292 | fit: 0.60793 |  sec: 0.026
INFO:brescal:[  3] LL: -2706.598 | fit: 0.61562 |  sec: 0.021
INFO:brescal:[  4] LL: -2736.216 | fit: 0.61561 |  sec: 0.019
INFO:brescal:[  5] LL: -2669.917 | fit: 0.62856 |  sec: 0.022
INFO:brescal:[  6] LL: -2669.637 | fit: 0.62725 |  sec: 0.022
INFO:brescal:[  7] LL: -2615.053 | fit: 0.63538 |  sec: 0.019
INFO:brescal:[  8] LL: -2580.257 | fit: 0.64210 |  sec: 0.022
INFO:brescal:[  9] LL: -2599.580 | fit: 0.63955 |  sec: 0.020


In [7]:
csr_X = list()
for k in range(n_relation):
    csr_X.append(csr_matrix(trainX[k]))
    
E, R, f, itr, exectimes = rescal.rescal_als(csr_X, n_dim)

INFO:RESCAL:[  0] fit: 0.61358 | delta: 6.1e-01 | secs: 0.00677
INFO:RESCAL:[  1] fit: 0.63865 | delta: 2.5e-02 | secs: 0.00893
INFO:RESCAL:[  2] fit: 0.65052 | delta: 1.2e-02 | secs: 0.00809
INFO:RESCAL:[  3] fit: 0.65754 | delta: 7.0e-03 | secs: 0.00949
INFO:RESCAL:[  4] fit: 0.66245 | delta: 4.9e-03 | secs: 0.00865
INFO:RESCAL:[  5] fit: 0.66666 | delta: 4.2e-03 | secs: 0.01015
INFO:RESCAL:[  6] fit: 0.67081 | delta: 4.1e-03 | secs: 0.01077
INFO:RESCAL:[  7] fit: 0.67490 | delta: 4.1e-03 | secs: 0.00922
INFO:RESCAL:[  8] fit: 0.67845 | delta: 3.5e-03 | secs: 0.00944
INFO:RESCAL:[  9] fit: 0.68103 | delta: 2.6e-03 | secs: 0.00663
INFO:RESCAL:[ 10] fit: 0.68264 | delta: 1.6e-03 | secs: 0.00796
INFO:RESCAL:[ 11] fit: 0.68356 | delta: 9.2e-04 | secs: 0.00994
INFO:RESCAL:[ 12] fit: 0.68405 | delta: 5.0e-04 | secs: 0.00974
INFO:RESCAL:[ 13] fit: 0.68432 | delta: 2.6e-04 | secs: 0.00962
INFO:RESCAL:[ 14] fit: 0.68445 | delta: 1.4e-04 | secs: 0.01009
INFO:RESCAL:[ 15] fit: 0.68453 | delta: 

In [8]:
_bX = model._reconstruct()

print("BayesRESCAL:Element-wise squared error on training data: %.3f" %(np.sum((trainX-_bX)**2)))
print("BayesRESCAL:Element-wise squared error on test data: %.3f\n" %(np.sum((X-_bX)[trainX==0]**2)))

_X = np.zeros_like(X)
for k in range(n_relation):
    _X[k] = np.dot(np.dot(E, R[k]), E.T)
    
print("RESCAL:Element-wise squared error on training data: %.3f" %(np.sum((trainX-_X)**2)))
print("RESCAL:Element-wise squared error on test data: %.3f" %(np.sum((X-_X)[trainX==0]**2)))

BayesRESCAL:Element-wise squared error on training data: 4834.098
BayesRESCAL:Element-wise squared error on test data: 11758.921

RESCAL:Element-wise squared error on training data: 4230.879
RESCAL:Element-wise squared error on test data: 11964.977


## Fit

Fit $= 1 - A / B$

$A = \sum_{i,k,j}(x_{i,k,j} - \hat{x}_{i,k,j})^2 \quad\quad$ where $\hat{x}$ is a predicted value of triple $(i,k,j)$. This is the sum of squared error.

$B = \sum_{i,k,j}(x_{i,k,j}) \quad\quad\quad\quad$ = Sum of squared datapoints.

$A$ is the objective function of RESCAL

In [9]:
A = np.sum((trainX-_X)**2)
B = np.sum(trainX**2)
fit = 1.-A/B
print(fit)

0.684526131412


## Control variance of observed/unobserved data

Placing different variances (var_x) on observed/unobserved points will improve the factorisation performance. (Implicit feedback)

Variance of observed elements (```obs_var```) = 1

Variance of unboserved elements (```unobs_var```) = 10000 (=almost do not care about unobserved data points)

In [10]:
model = BayesianRescal(n_dim, var_e=var_e, var_x=var_x, var_r=var_r, controlled_var=True, obs_var=1., unobs_var=10000.)
model.fit(trainX, max_iter=10)
_cX = model._reconstruct()

print("BayesRESCAL:Element-wise squared error on training data: %.3f" %(np.sum((trainX-_bX)**2)))
print("BayesRESCAL:Element-wise squared error on test data: %.3f\n" %(np.sum((X-_bX)[trainX==0]**2)))

print("RESCAL:Element-wise squared error on training data: %.3f" %(np.sum((trainX-_X)**2)))
print("RESCAL:Element-wise squared error on test data: %.3f\n" %(np.sum((X-_X)[trainX==0]**2)))

print("CV_BayesRESCAL:Element-wise squared error on training data: %.3f" %(np.sum((trainX-_cX)**2)))
print("CV_BayesRESCAL:Element-wise squared error on test data: %.3f" %(np.sum((X-_cX)[trainX==0]**2)))

INFO:brescal:[  0] LL: -5185.936 | fit: 0.24084 |  sec: 0.038
INFO:brescal:[  1] LL: -3302.678 | fit: 0.51882 |  sec: 0.045
INFO:brescal:[  2] LL: -2936.063 | fit: 0.57449 |  sec: 0.033
INFO:brescal:[  3] LL: -2825.958 | fit: 0.59436 |  sec: 0.042
INFO:brescal:[  4] LL: -2838.909 | fit: 0.59877 |  sec: 0.037
INFO:brescal:[  5] LL: -2848.886 | fit: 0.60563 |  sec: 0.040
INFO:brescal:[  6] LL: -2928.899 | fit: 0.60557 |  sec: 0.038
INFO:brescal:[  7] LL: -3087.699 | fit: 0.60195 |  sec: 0.039
INFO:brescal:[  8] LL: -3195.070 | fit: 0.60407 |  sec: 0.036
INFO:brescal:[  9] LL: -3360.056 | fit: 0.60046 |  sec: 0.038


BayesRESCAL:Element-wise squared error on training data: 4834.098
BayesRESCAL:Element-wise squared error on test data: 11758.921

RESCAL:Element-wise squared error on training data: 4230.879
RESCAL:Element-wise squared error on test data: 11964.977

CV_BayesRESCAL:Element-wise squared error on training data: 5358.326
CV_BayesRESCAL:Element-wise squared error on test data: 10939.803


### Controlled variance approach reduced the squared error on test data